<a href="https://colab.research.google.com/github/18521144/Emotion_Detection/blob/master/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DOWNLOAD DỮ LIỆU TỪ KAGGLE**

LINK: https://www.kaggle.com/msambare/fer2013

In [ ]:
!pip install -q kaggle

from google.colab import files

files.upload()

In [ ]:
!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d msambare/fer2013

In [ ]:
!unzip fer2013.zip

**IMPORT CÁC THƯ VIỆN CẦN THIẾT**


In [ ]:
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator



**CHUẨN BỊ BỘ DỮ LIỆU TRAIN, TEST**

In [ ]:
train_dir = '/content/train'
val_dir = '/content/test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

**XÂY DỰNG MẠNG TÍNH CHẬP**

In [ ]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

**TRAIN MODEL**

In [ ]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
448/448 [==============================] - 397s 883ms/step - loss: 1.8284 - accuracy: 0.2516 - val_loss: 1.7223 - val_accuracy: 0.3486
Epoch 2/50
448/448 [==============================] - 397s 886ms/step - loss: 1.6633 - accuracy: 0.3507 - val_loss: 1.5522 - val_accuracy: 0.4058
Epoch 3/50
448/448 [==============================] - 397s 886ms/step - loss: 1.5456 - accuracy: 0.4074 - val_loss: 1.4850 - val_accuracy: 0.4360
Epoch 4/50
448/448 [==============================] - 392s 875ms/step - loss: 1.4860 - accuracy: 0.4237 - val_loss: 1.4162 - val_accuracy: 0.4621
Epoch 5/50
448/448 [==============================] - 393s 877ms/step - loss: 1.4274 - accuracy: 0.4514 - val_loss: 1.3702 - val_accuracy: 0.4771
Epoch 6/50
448/448 [==============================] - 394s 879ms/step - loss: 1.3723 - accuracy: 0.4774 - val_loss: 1.3269 - val_accuracy: 0.4905
Epoch 7/50
448/448 [==============================] - 392s 875ms/step - loss: 1.3219 - accuracy: 0.4988 - val_loss: 1.2928 -

**SAVE MODEL**

In [ ]:
emotion_model.save_weights('model.h5')
